In [24]:
 """This function will use the csv library to save the qualifying data as a file"""
    
def save_csv(outhput_path): 
    with open(outhput_path, "w") as csvfile:
        file = []
        outhput_path = Path("qualifying_loan.csv")
        
        csvwriter = csv.write(csvfile, outthput_path) 
        
        csvwriter.writerow(header)

        for loan in csvwriter:
            csvwriter.writerow(loan.values())
        
        return file
    
     
        